In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import os
print("Tensorflow version " + tf.__version__)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

Tensorflow version 2.8.0
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.112.33.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.112.33.250:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [40]:
import numpy as np
import keras
import time
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback
from keras.layers import Dense,Reshape,Input,BatchNormalization,Dropout,concatenate,Flatten,Lambda,Resizing,CuDNNLSTM,concatenate,Layer,LSTM
from keras import  Model,layers
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split

In [ ]:
def find_removable(n,batch_size=64):
      return n%batch_size

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/OAD/OAD/Vincent/BASE/30/'
# base_path = '/content/drive/MyDrive/Colab Notebooks/NTU/Vincent/ntu_3cam/'
input_shape= (25,30, 3)

train_x = np.load(base_path+'train_x.npy')
test_x = np.load(base_path+'test_x.npy')
train_y = np.load(base_path+'train_y.npy')
test_y = np.load(base_path+'test_y.npy')

batch_size=64

n = find_removable(len(train_x),batch_size)
train_x = train_x[:-n]
train_y = train_y[:-n]

print('lost',n)

train = (train_x,train_y)
test = (test_x,test_y)

lost 31


In [53]:
with strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  inp=Input(np.asarray(input_shape))
  output = Resizing(64, 64)(inp)
  model = VGG16(include_top=False,input_shape=(64,64,3))
  for layer in model.layers:
      layer.trainable = True
  output = model(output)
  output = Flatten()(output)
  output = Dense(2048)(output)
  output = BatchNormalization()(output)
  output = Dense(10, activation='softmax',kernel_regularizer='l1')(output)
  model = Model(inp, output)
  model.summary()
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=optimizer,
                metrics=['accuracy'])

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 25, 30, 3)]       0         
                                                                 
 resizing_20 (Resizing)      (None, 64, 64, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten_17 (Flatten)        (None, 2048)              0         
                                                                 
 dense_34 (Dense)            (None, 2048)              4196352   
                                                                 
 batch_normalization_10 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                          

In [54]:
filepath ="oad.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
callbacks = [checkpoint,EarlyStopping(monitor='val_accuracy', patience=100,)]
debut = time.time()
history=model.fit(train[0], train[1],
              epochs=200,
              batch_size=batch_size,
              validation_data=(test[0],test[1]),
              callbacks=callbacks,
              verbose=1)
fin = time.time()
print("Temps total : " + str(fin-debut) + "ms")
model = keras.models.load_model('oad.hdf5')
keras.models.save_model(model, base_path+'_vgg16_64.h5')
print(model.evaluate(test[0],test[1]))

Epoch 1/200
186/186 [==============================] - 15s 57ms/step - loss: 4.6514 - accuracy: 0.7745 - val_loss: 4.7400 - val_accuracy: 0.4617
Epoch 2/200
186/186 [==============================] - 7s 37ms/step - loss: 1.5107 - accuracy: 0.9752 - val_loss: 1.6521 - val_accuracy: 0.7032
Epoch 3/200
186/186 [==============================] - 7s 37ms/step - loss: 0.5762 - accuracy: 0.9922 - val_loss: 0.8327 - val_accuracy: 0.8729
Epoch 4/200
186/186 [==============================] - 7s 38ms/step - loss: 0.4510 - accuracy: 0.9945 - val_loss: 0.7338 - val_accuracy: 0.8735
Epoch 5/200
186/186 [==============================] - 7s 37ms/step - loss: 0.4238 - accuracy: 0.9941 - val_loss: 0.5899 - val_accuracy: 0.9221
Epoch 6/200
186/186 [==============================] - 7s 35ms/step - loss: 0.3950 - accuracy: 0.9976 - val_loss: 0.6922 - val_accuracy: 0.8835
Epoch 7/200
186/186 [==============================] - 6s 34ms/step - loss: 0.3823 - accuracy: 0.9981 - val_loss: 0.5549 - val_accuracy